In [ ]:
%matplotlib inline

# Notebook summary

The purpose of this notebook is simply to clean up some of the files that were created in `Compile_data.ipynb`. The output of that notebook was a set of `.tsv` files organized into folders within `../Data/` for both host and virus genomes. Those `.tsv` files (potentially) contained some bugs so we're going to run some filters/tests and store cleaned versions. 

# Imports

The only custom import here is iCUB (a codon usage bias calculator) that can be found here:
https://github.com/amarallab/iCUB

In [1]:
import pandas as pd
import glob
import sys
sys.path.append('../../iCUB/') ###iCUB needs to be installed for this to work
import iCUB

# Custom functions

Despite some similarities, there are a few crucial differences between the host and virus `.tsv` files that dictate separate cleaning functions.

In [5]:
def common_cleaning(df):
    
    df = df[df['upstream_sequence'].isnull()==False]
    df = df[df['coding_sequence'].isnull()==False]
    
    ###Add a codon usage bias and GC percent columns
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq) != nt_seq.count('A') + nt_seq.count('T') + nt_seq.count('C') + nt_seq.count('G'):
            continue
        if len(nt_seq)%3 != 0:
            continue
        df.at[index, 'iCUB'] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
        #
        df.at[index, 'GC_cds'] = (nt_seq.count('G') + nt_seq.count('C')) / len(nt_seq)
        #
        up_seq = df.at[index, 'upstream_sequence']
        df.at[index, 'GC_upstream'] = (up_seq.count('G') + up_seq.count('C')) / len(up_seq)


    
    ###Remove genes for which we were unable to calculate codon usage bias or RBS energies
    df = df[df['iCUB'].isnull()==False]
    df = df[df['GC_cds'].isnull()==False]
    df = df[df['GC_upstream'].isnull()==False]
    df = df[df['RBS_energy'].isnull()==False]
    df = df[df['RBS_energy_upstream'].isnull()==False]
    return df

def clean_host_tsv(df):
    """
    WRITE A BRIEF PURPOSE/SUMMARY
    
    Development notes: I could also think about testing for stop codons within coding sequences
    and filtering accordingly. Also a way to test for possible non-standard genetic code usage
    
    Input/s:
        df - a pandas dataframe with numeric indices, outputted from Compile_data.ipynb and read
                back in
        
    Output/s:
        df - a clean version of the dataframe with one new column (iCUB) and (potentially)
                several rows removed
    """
    df = df.reset_index(drop = True)

    ###Filter out possible prophage genes by removing anything involving the word phage
    ###Numerous possibilities/ways to do this and not all genomes might have any decent
    ###descriptions in the qualifiers.
    filter_word = 'phage'
    df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    df = common_cleaning(df)
    
    ###Now ensure that each locus tag is only used once and when in doubt remove them both
    df['locus_tag'] = df['qualifiers'].str.split('locus_tag=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ['locus_tag'], keep = False)
    return df

def clean_virus_tsv(df):
    """
    This is the same basic structure as the "clean_host_tsv" function. 
    
    Input/s:
        df - a pandas dataframe with numeric indices, outputted from Compile_data.ipynb and read
                back in
        
    Output/s:
        df - a clean version of the dataframe with one new column (iCUB) and (potentially)
                several rows removed
                
    """  
    df = df.reset_index(drop = True)
    
    df = common_cleaning(df)

    ###Now ensure that each viral_id tag is only used once (and when in doubt remove them BOTH)
    df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["viral_id"], keep = False)
    return df



# First, test the translation table assumption

In the future, if this analysis is extended to any genome where the `transl_table` qualifier is **not** 11 there will need to be extensive code re-factoring, starting with re-annotating the viral genomes to account for this fact

In [4]:
for host_tsv_file in glob.glob('../Data/host_genomes/' + '*.tsv')[:]:
    if '.clean.' in host_tsv_file:
        continue
    print(host_tsv_file)
    df = pd.read_csv(host_tsv_file, sep = "\t", index_col = 0)
    df['transl_table'] = df['qualifiers'].str.split('transl_table=', n=1, expand=True)[1]\
                        .str.split(';', n=1, expand=True)[0]
    print(df.shape[0], list(df['transl_table'].value_counts().items()))

../Data/host_genomes/28450.tsv
5727 [('11', 5727)]
../Data/host_genomes/1590.tsv
3013 [('11', 3013)]
../Data/host_genomes/562.tsv
4379 [('11', 4379)]
../Data/host_genomes/357276.tsv
4343 [('11', 4343)]
../Data/host_genomes/657318.tsv
3294 [('11', 3294)]
../Data/host_genomes/573.tsv
5316 [('11', 5316)]
../Data/host_genomes/1280.tsv
2767 [('11', 2767)]
../Data/host_genomes/305.tsv
3466 [('11', 3466)]
../Data/host_genomes/435591.tsv
3979 [('11', 3979)]
../Data/host_genomes/470.tsv
4327 [('11', 4327)]
../Data/host_genomes/287.tsv
5573 [('11', 5573)]
../Data/host_genomes/1314.tsv
1693 [('11', 1690)]
../Data/host_genomes/36809.tsv
4920 [('11', 4920)]
../Data/host_genomes/717959.tsv
3110 [('11', 3110)]
../Data/host_genomes/90371.tsv
4447 [('11', 4447)]
../Data/host_genomes/1639.tsv
2867 [('11', 2867)]
../Data/host_genomes/1428.tsv
5117 [('11', 5117)]


# Run cleaning function on all host `.tsv` files

This isn't fast but it's also not optimized for speed. At the moment I am only looking at < 20 host files and this code only needs to be run once so speed is taking a backseat to thoroughness. Ergo, for loops, etc. As it stands, my guess is that this cell takes ~45 mins or so to run

In [ ]:
for host_tsv_file in glob.glob('../Data/host_genomes/' + '*.tsv'):
    if '.clean.' in host_tsv_file:
        continue
    print(host_tsv_file)
    ###
    df = pd.read_csv(host_tsv_file, sep = "\t", index_col = 0)
    initial_shape = df.shape
    df = clean_host_tsv(df)
    final_shape = df.shape
    ###
    gene_ratio = final_shape[0]/initial_shape[0]
    print(gene_ratio)
    if gene_ratio <= 0.8:
        break
    ###
    clean_tsv_loc = host_tsv_file.replace(".tsv",".clean.tsv")
    df.to_csv(clean_tsv_loc, sep = "\t")

# Run cleaning function on all phage `.tsv` files

Currently printing a lot but realistically there aren't to many files in my current database so it doesn't seem to be a problem to monitor ongoing progress in this manner. This runs fairly quickly, ~30 mins I think.

In [ ]:
for virus_folder in glob.glob('../Data/*_rep_viruses/'):
    print('###', virus_folder)
    for virus_tsv_file in glob.glob(virus_folder + '*.tsv'):
        if ".clean." in virus_tsv_file:
            continue
        print(virus_tsv_file)
        ###
        df = pd.read_csv(virus_tsv_file, sep = "\t", index_col = 0)
        initial_shape = df.shape
        df = clean_virus_tsv(df)
        final_shape = df.shape
        ###
        gene_ratio = final_shape[0]/initial_shape[0]
        if gene_ratio <= 0.8:
            print('Strange case', gene_ratio)
            break
        ###
        clean_tsv_loc = virus_tsv_file.replace(".tsv",".clean.tsv")
        df.to_csv(clean_tsv_loc, sep = "\t")